# Counterfactual with Reinforcement Learning (CFRL) on Adult Census

This method is described in [Model-agnostic and Scalable Counterfactual Explanations via Reinforcement Learning](https://arxiv.org/abs/2106.02597) and can generate counterfactual instances for any black-box model. The usual optimization procedure is transformed into a learnable process allowing to generate batches of counterfactual instances in a single forward pass even for high dimensional data. The training pipeline is model-agnostic and relies only on prediction feedback by querying the black-box model. Furthermore, the method allows target and feature conditioning. 

**We exemplify the use case for the TensorFlow backend. This means that all models: the autoencoder, the actor and the critic are TensorFlow models. Our implementation supports PyTorch backend as well.**

CFRL uses [Deep Deterministic Policy Gradient (DDPG)](https://arxiv.org/abs/1509.02971) by interleaving a state-action function approximator called critic, with a learning an approximator called actor to predict the optimal action. The method assumes that the critic is differentiable with respect to the action argument, thus allowing to optimize the actor's parameters efficiently through gradient-based methods.

The DDPG algorithm requires two separate networks, an actor $\mu$ and a critic $Q$. Given the encoded representation of the input instance $z = enc(x)$, the model prediction $y_M$, the target prediction
$y_T$ and the conditioning vector $c$, the actor outputs the counterfactual’s latent representation $z_{CF} = \mu(z, y_M, y_T, c)$. The decoder then projects the embedding $z_{CF}$ back to the original input space,
followed by optional post-processing.

The training step consists of simultaneously optimizing the actor and critic networks. The critic regresses on the reward $R$ determined by the model prediction, while the actor maximizes the critic’s output for the given instance through $L_{max}$. The actor also minimizes two objectives to encourage the generation of sparse, in-distribution counterfactuals. The sparsity loss $L_{sparsity}$ operates on the decoded counterfactual $x_{CF}$ and combines the $L_1$ loss over the standardized numerical features and the $L_0$ loss over the categorical ones. The consistency loss $L_{consist}$ aims to encode the counterfactual $x_{CF}$ back to the same latent representation where it was decoded from and helps to produce in-distribution counterfactual instances. Formally, the actor's loss can be written as:
$L_{actor} = L_{max} + \lambda_{1}L_{sparsity} + \lambda_{2}L_{consistency}$

This example will use the [xgboost](https://github.com/dmlc/xgboost) library, which can be installed with:

In [ ]:
!pip install xgboost

In [1]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy
from typing import List, Tuple, Dict, Callable

import tensorflow as tf
import tensorflow.keras as keras

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from alibi.explainers import CounterfactualRLTabular, CounterfactualRL
from alibi.datasets import fetch_adult
from alibi.models.tensorflow.autoencoder import HeAE
from alibi.models.tensorflow.actor_critic import Actor, Critic
from alibi.models.tensorflow.cfrl_models import ADULTEncoder, ADULTDecoder
from alibi.explainers.cfrl_base import Callback
from alibi.explainers.backends.cfrl_tabular import get_he_preprocessor, get_statistics, \
    get_conditional_vector, apply_category_mapping

### Load Adult Census Dataset

In [2]:
# Fetch adult dataset
adult = fetch_adult()

# Separate columns in numerical and categorical.
categorical_names = [adult.feature_names[i] for i in adult.category_map.keys()]
categorical_ids = list(adult.category_map.keys())

numerical_names = [name for i, name in enumerate(adult.feature_names) if i not in adult.category_map.keys()]
numerical_ids = [i for i in range(len(adult.feature_names)) if i not in adult.category_map.keys()]

# Split data into train and test
X, Y = adult.data, adult.target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=13)

### Train black-box classifier

In [3]:
# Define numerical standard scaler.
num_transf = StandardScaler()

# Define categorical one-hot encoder.
cat_transf = OneHotEncoder(
    categories=[range(len(x)) for x in adult.category_map.values()],
    handle_unknown="ignore"
)

# Define column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", cat_transf, categorical_ids),
        ("num", num_transf, numerical_ids),
    ],
    sparse_threshold=0
)

In [4]:
# Fit preprocessor.
preprocessor.fit(X_train)

# Preprocess train and test dataset.
X_train_ohe = preprocessor.transform(X_train)
X_test_ohe = preprocessor.transform(X_test)

In [5]:
# Select one of the below classifiers.
clf = XGBClassifier(min_child_weight=0.5, max_depth=3, gamma=0.2)
# clf = LogisticRegression(C=10)
# clf = DecisionTreeClassifier(max_depth=10, min_samples_split=5)
# clf = RandomForestClassifier(max_depth=15, min_samples_split=10, n_estimators=50)

# Fit the classifier.
clf.fit(X_train_ohe, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.2, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=0.5,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

### Define the predictor (black-box)

Now that we've trained the classifier, we can define the black-box model. Note that the output of the black-box is a distribution which can be either a soft-label distribution (probabilities/logits for each class) or a hard-label distribution (one-hot encoding). Internally, CFRL takes the `argmax`. Moreover the output **DOES NOT HAVE TO BE DIFFERENTIABLE**.

In [6]:
# Define prediction function.
predictor = lambda x: clf.predict_proba(preprocessor.transform(x))

In [7]:
# Compute accuracy.
acc = accuracy_score(y_true=Y_test, y_pred=predictor(X_test).argmax(axis=1))
print("Accuracy: %.3f" % acc)

Accuracy: 0.870


### Define and train autoencoder

Instead of directly modelling the perturbation vector in the potentially high-dimensional input space, we first train an autoencoder. The weights of the encoder are frozen and the actor applies the
counterfactual perturbations in the latent space of the encoder. The pre-trained decoder maps the counterfactual embedding back to the input feature space. 

The autoencoder follows a standard design. The model is composed from two submodules, the encoder and the decoder. The forward pass consists of passing the input to the encoder, obtain the input embedding and pass the embedding through the decoder.

```python
class HeAE(keras.Model):
    def __init__(self, encoder: keras.Model, decoder: keras.Model, **kwargs) -> None:
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, x: tf.Tensor, **kwargs):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat
```

The heterogeneous variant used in this example uses an additional type checking to ensure that the output of the decoder is a list of tensors.

Heterogeneous dataset require special treatment. In this work we modeled the numerical features by normal distributions with constant standard deviation and categorical features by categorical distributions. Due to the choice of feature modeling, some numerical features can end up having different types than the original numerical features. For example, a feature like `Age` having the type of `int` can become a `float` due to the autoencoder reconstruction (e.g., `Age=26 -> Age=26.3`). This behavior can be undesirable. Thus we performed casting when process the output of the autoencoder (decoder component).

In [8]:
# Define attribute types, required for datatype conversion.
feature_types = {"Age": int, "Capital Gain": int, "Capital Loss": int, "Hours per week": int}

# Define data preprocessor and inverse preprocessor. The invers preprocessor include datatype conversions.
heae_preprocessor, heae_inv_preprocessor = get_he_preprocessor(X=X_train,
                                                               feature_names=adult.feature_names,
                                                               category_map=adult.category_map,
                                                               feature_types=feature_types)

# Define trainset
trainset_input = heae_preprocessor(X_train).astype(np.float32)
trainset_outputs = {
    "output_1": trainset_input[:, :len(numerical_ids)]
}

for i, cat_id in enumerate(categorical_ids):
    trainset_outputs.update({
        f"output_{i+2}": X_train[:, cat_id]
    })
    
trainset = tf.data.Dataset.from_tensor_slices((trainset_input, trainset_outputs))
trainset = trainset.shuffle(1024).batch(128, drop_remainder=True)

In [9]:
# Define autoencoder path and create dir if it doesn't exist.
heae_path = os.path.join("tensorflow", "ADULT_autoencoder")
if not os.path.exists(heae_path):
    os.makedirs(heae_path)

# Define constants.
EPOCHS = 50              # epochs to train the autoencoder
HIDDEN_DIM = 128         # hidden dimension of the autoencoder
LATENT_DIM = 15          # define latent dimension

# Define output dimensions.
OUTPUT_DIMS = [len(numerical_ids)]
OUTPUT_DIMS += [len(adult.category_map[cat_id]) for cat_id in categorical_ids]

# Define the heterogeneous auto-encoder.
heae = HeAE(encoder=ADULTEncoder(hidden_dim=HIDDEN_DIM, latent_dim=LATENT_DIM),
            decoder=ADULTDecoder(hidden_dim=HIDDEN_DIM, output_dims=OUTPUT_DIMS))

# Define loss functions.
he_loss = [keras.losses.MeanSquaredError()]
he_loss_weights = [1.]

# Add categorical losses.
for i in range(len(categorical_names)):
    he_loss.append(keras.losses.SparseCategoricalCrossentropy(from_logits=True))
    he_loss_weights.append(1./len(categorical_names))

# Define metrics.
metrics = {}
for i, cat_name in enumerate(categorical_names):
    metrics.update({f"output_{i+2}": keras.metrics.SparseCategoricalAccuracy()})
    
# Compile model.
heae.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
             loss=he_loss,
             loss_weights=he_loss_weights,
             metrics=metrics)

if len(os.listdir(heae_path)) == 0:
    # Fit and save autoencoder.
    heae.fit(trainset, epochs=EPOCHS)
    heae.save(heae_path, save_format="tf")
else:
    # Load the model.
    heae = keras.models.load_model(heae_path, compile=False)

Epoch 1/50
203/203 [==============================] - 3s 6ms/step - loss: 1.1241 - output_1_loss: 0.2301 - output_2_loss: 1.0748 - output_3_loss: 1.2238 - output_4_loss: 0.6933 - output_5_loss: 1.6187 - output_6_loss: 0.9102 - output_7_loss: 0.5906 - output_8_loss: 0.3292 - output_9_loss: 0.7112 - output_2_sparse_categorical_accuracy: 0.6824 - output_3_sparse_categorical_accuracy: 0.5632 - output_4_sparse_categorical_accuracy: 0.7414 - output_5_sparse_categorical_accuracy: 0.4145 - output_6_sparse_categorical_accuracy: 0.6954 - output_7_sparse_categorical_accuracy: 0.8366 - output_8_sparse_categorical_accuracy: 0.8535 - output_9_sparse_categorical_accuracy: 0.8744
Epoch 2/50
203/203 [==============================] - 1s 6ms/step - loss: 0.3998 - output_1_loss: 0.0378 - output_2_loss: 0.4761 - output_3_loss: 0.4630 - output_4_loss: 0.1982 - output_5_loss: 0.6726 - output_6_loss: 0.3669 - output_7_loss: 0.2860 - output_8_loss: 0.0919 - output_9_loss: 0.3407 - output_2_sparse_categorical_

203/203 [==============================] - 1s 6ms/step - loss: 0.0461 - output_1_loss: 0.0080 - output_2_loss: 0.0488 - output_3_loss: 0.0479 - output_4_loss: 0.0239 - output_5_loss: 0.0519 - output_6_loss: 0.0292 - output_7_loss: 0.0315 - output_8_loss: 0.0070 - output_9_loss: 0.0647 - output_2_sparse_categorical_accuracy: 0.9871 - output_3_sparse_categorical_accuracy: 0.9879 - output_4_sparse_categorical_accuracy: 0.9944 - output_5_sparse_categorical_accuracy: 0.9871 - output_6_sparse_categorical_accuracy: 0.9938 - output_7_sparse_categorical_accuracy: 0.9916 - output_8_sparse_categorical_accuracy: 0.9985 - output_9_sparse_categorical_accuracy: 0.9817
Epoch 14/50
203/203 [==============================] - 1s 6ms/step - loss: 0.0424 - output_1_loss: 0.0074 - output_2_loss: 0.0450 - output_3_loss: 0.0443 - output_4_loss: 0.0222 - output_5_loss: 0.0476 - output_6_loss: 0.0268 - output_7_loss: 0.0291 - output_8_loss: 0.0064 - output_9_loss: 0.0587 - output_2_sparse_categorical_accuracy: 

203/203 [==============================] - 1s 6ms/step - loss: 0.0210 - output_1_loss: 0.0045 - output_2_loss: 0.0209 - output_3_loss: 0.0210 - output_4_loss: 0.0109 - output_5_loss: 0.0228 - output_6_loss: 0.0121 - output_7_loss: 0.0137 - output_8_loss: 0.0031 - output_9_loss: 0.0274 - output_2_sparse_categorical_accuracy: 0.9947 - output_3_sparse_categorical_accuracy: 0.9949 - output_4_sparse_categorical_accuracy: 0.9977 - output_5_sparse_categorical_accuracy: 0.9945 - output_6_sparse_categorical_accuracy: 0.9975 - output_7_sparse_categorical_accuracy: 0.9964 - output_8_sparse_categorical_accuracy: 0.9992 - output_9_sparse_categorical_accuracy: 0.9933
Epoch 26/50
203/203 [==============================] - 1s 6ms/step - loss: 0.0200 - output_1_loss: 0.0044 - output_2_loss: 0.0197 - output_3_loss: 0.0199 - output_4_loss: 0.0106 - output_5_loss: 0.0213 - output_6_loss: 0.0115 - output_7_loss: 0.0133 - output_8_loss: 0.0027 - output_9_loss: 0.0259 - output_2_sparse_categorical_accuracy: 

203/203 [==============================] - 1s 6ms/step - loss: 0.0117 - output_1_loss: 0.0032 - output_2_loss: 0.0098 - output_3_loss: 0.0103 - output_4_loss: 0.0065 - output_5_loss: 0.0106 - output_6_loss: 0.0062 - output_7_loss: 0.0072 - output_8_loss: 0.0015 - output_9_loss: 0.0160 - output_2_sparse_categorical_accuracy: 0.9980 - output_3_sparse_categorical_accuracy: 0.9977 - output_4_sparse_categorical_accuracy: 0.9987 - output_5_sparse_categorical_accuracy: 0.9979 - output_6_sparse_categorical_accuracy: 0.9990 - output_7_sparse_categorical_accuracy: 0.9981 - output_8_sparse_categorical_accuracy: 0.9999 - output_9_sparse_categorical_accuracy: 0.9963
Epoch 38/50
203/203 [==============================] - 1s 6ms/step - loss: 0.0114 - output_1_loss: 0.0032 - output_2_loss: 0.0093 - output_3_loss: 0.0098 - output_4_loss: 0.0063 - output_5_loss: 0.0105 - output_6_loss: 0.0060 - output_7_loss: 0.0070 - output_8_loss: 0.0016 - output_9_loss: 0.0154 - output_2_sparse_categorical_accuracy: 

203/203 [==============================] - 1s 6ms/step - loss: 0.0072 - output_1_loss: 0.0023 - output_2_loss: 0.0052 - output_3_loss: 0.0056 - output_4_loss: 0.0039 - output_5_loss: 0.0055 - output_6_loss: 0.0038 - output_7_loss: 0.0044 - output_8_loss: 0.0010 - output_9_loss: 0.0096 - output_2_sparse_categorical_accuracy: 0.9991 - output_3_sparse_categorical_accuracy: 0.9990 - output_4_sparse_categorical_accuracy: 0.9994 - output_5_sparse_categorical_accuracy: 0.9995 - output_6_sparse_categorical_accuracy: 0.9994 - output_7_sparse_categorical_accuracy: 0.9991 - output_8_sparse_categorical_accuracy: 0.9999 - output_9_sparse_categorical_accuracy: 0.9982
Epoch 50/50
203/203 [==============================] - 1s 6ms/step - loss: 0.0076 - output_1_loss: 0.0025 - output_2_loss: 0.0052 - output_3_loss: 0.0057 - output_4_loss: 0.0045 - output_5_loss: 0.0056 - output_6_loss: 0.0039 - output_7_loss: 0.0046 - output_8_loss: 0.0013 - output_9_loss: 0.0098 - output_2_sparse_categorical_accuracy: 

### Counterfactual with Reinforcement Learning

In [10]:
# Define constants
COEFF_SPARSITY = 0.5               # sparisty coefficient
COEFF_CONSISTENCY = 0.5            # consisteny coefficient
TRAIN_STEPS = 5000                # number of training steps -> consider increasing the number of steps
BATCH_SIZE = 100                   # batch size

#### Define dataset specific attributes and constraints

A desirable property of a method for generating counterfactuals is to allow feature conditioning. Real-world datasets usually include immutable features such as `Sex` or `Race`, which should remain unchanged throughout the counterfactual search procedure. Similarly, a numerical feature such as `Age` should only increase for a counterfactual to be actionable. 

In [11]:
# Define immutable features.
immutable_features = ['Marital Status', 'Relationship', 'Race', 'Sex']

# Define ranges. This means that the `Age` feature can not decrease.
ranges = {'Age': [0.0, 1.0]}

#### Define and fit the explainer

In [12]:
explainer = CounterfactualRLTabular(predictor=predictor,
                                    encoder=heae.encoder,
                                    decoder=heae.decoder,
                                    latent_dim=LATENT_DIM,
                                    encoder_preprocessor=heae_preprocessor,
                                    decoder_inv_preprocessor=heae_inv_preprocessor,
                                    coeff_sparsity=COEFF_SPARSITY,
                                    coeff_consistency=COEFF_CONSISTENCY,
                                    category_map=adult.category_map,
                                    feature_names=adult.feature_names,
                                    ranges=ranges,
                                    immutable_features=immutable_features,
                                    train_steps=TRAIN_STEPS,
                                    batch_size=BATCH_SIZE,
                                    backend="tensorflow")

In [13]:
# Fit the explainer.
explainer = explainer.fit(X=X_train)

100%|██████████| 5000/5000 [02:59<00:00, 27.78it/s]


#### Test explainer

In [14]:
# Select some positive examples.
X_positive = X_test[np.argmax(predictor(X_test), axis=1) == 1]

X = X_positive[:1000]
Y_t = np.array([0])
C = [{"Age": [0, 20], "Workclass": ["State-gov", "?", "Local-gov"]}]

In [15]:
# Generate counterfactual instances.
explanation = explainer.explain(X, Y_t, C)

100%|██████████| 10/10 [00:00<00:00, 44.71it/s]


In [16]:
# Concat labels to the original instances.
orig = np.concatenate(
    [explanation.data['orig']['X'], explanation.data['orig']['class']],
    axis=1
)

# Concat labels to the counterfactual instances.
cf = np.concatenate(
    [explanation.data['cf']['X'], explanation.data['cf']['class']],
    axis=1
)

# Define new feature names and category map by including the label.
feature_names = adult.feature_names + ["Label"]
category_map = deepcopy(adult.category_map)
category_map.update({feature_names.index("Label"): adult.target_names})

# Replace label encodings with strings.
orig_pd = pd.DataFrame(
    apply_category_mapping(orig, category_map),
    columns=feature_names
)

cf_pd = pd.DataFrame(
    apply_category_mapping(cf, category_map),
    columns=feature_names
)

In [17]:
orig_pd.head(n=10)

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Label
0,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,7298,0,40,United-States,>50K
1,35,Private,High School grad,Married,White-Collar,Husband,White,Male,7688,0,50,United-States,>50K
2,39,State-gov,Masters,Married,Professional,Wife,White,Female,5178,0,38,United-States,>50K
3,44,Self-emp-inc,High School grad,Married,Sales,Husband,White,Male,0,0,50,United-States,>50K
4,44,Federal-gov,High School grad,Married,Admin,Husband,White,Male,0,0,40,United-States,>50K
5,39,Private,Bachelors,Separated,White-Collar,Not-in-family,White,Female,13550,0,50,United-States,>50K
6,45,Private,High School grad,Married,Blue-Collar,Husband,White,Male,0,1902,40,?,>50K
7,50,Private,Bachelors,Married,Professional,Husband,White,Male,0,0,50,United-States,>50K
8,29,Private,Bachelors,Married,White-Collar,Wife,White,Female,0,0,50,United-States,>50K
9,47,Private,Bachelors,Married,Professional,Husband,White,Male,0,0,50,United-States,>50K


In [18]:
cf_pd.head(n=10)

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Label
0,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,1063,0,39,United-States,<=50K
1,35,Private,High School grad,Married,Blue-Collar,Husband,White,Male,1420,0,48,United-States,<=50K
2,41,Local-gov,Dropout,Married,Blue-Collar,Wife,White,Female,1124,0,38,United-States,<=50K
3,44,Self-emp-inc,High School grad,Married,Blue-Collar,Husband,White,Male,518,0,48,United-States,<=50K
4,44,Federal-gov,High School grad,Married,Blue-Collar,Husband,White,Male,390,0,41,United-States,<=50K
5,39,Private,Dropout,Separated,White-Collar,Not-in-family,White,Female,1223,0,49,United-States,<=50K
6,45,Private,High School grad,Married,Blue-Collar,Husband,White,Male,1190,1998,39,United-States,<=50K
7,50,Private,High School grad,Married,Blue-Collar,Husband,White,Male,298,0,49,United-States,<=50K
8,29,Private,High School grad,Married,Sales,Wife,White,Female,422,0,50,United-States,<=50K
9,47,Private,High School grad,Married,Blue-Collar,Husband,White,Male,293,0,49,United-States,<=50K


#### Diversity

In [19]:
# Generate counterfactual instances.
X = X_positive[0].reshape(1, -1)
explanation = explainer.explain(X=X, Y_t=Y_t, C=C, diversity=True, num_samples=100, batch_size=10)

11it [00:00, 33.91it/s]


In [20]:
# Concat label column.
orig = np.concatenate(
    [explanation.data['orig']['X'], explanation.data['orig']['class']],
    axis=1
)

cf = np.concatenate(
    [explanation.data['cf']['X'], explanation.data['cf']['class']],
    axis=1
)

# Transfrom label encodings to string.
orig_pd = pd.DataFrame(
    apply_category_mapping(orig, category_map),
    columns=feature_names,
)

cf_pd = pd.DataFrame(
    apply_category_mapping(cf, category_map),
    columns=feature_names,
)

In [21]:
orig_pd.head(n=5)

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Label
0,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,7298,0,40,United-States,>50K


In [22]:
cf_pd.head(n=5)

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Label
0,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,541,0,39,United-States,<=50K
1,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,642,0,39,United-States,<=50K
2,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,644,0,39,United-States,<=50K
3,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,678,0,39,United-States,<=50K
4,60,Private,High School grad,Married,Blue-Collar,Husband,White,Male,681,0,39,United-States,<=50K


### Logging

Logging is clearly important when dealing with deep learning models. Thus, we provide an interface to write custom callbacks for logging purposes after each training step which we defined [here](../api/alibi.explainers.cfrl_base.rst#alibi.explainers.cfrl_base.Callback). In the following cells we provide some example to log in **Weights and Biases**.

#### Logging reward callback

In [23]:
class RewardCallback(Callback):
    def __call__(self,
                 step: int, 
                 update: int, 
                 model: CounterfactualRL,
                 sample: Dict[str, np.ndarray],
                 losses: Dict[str, float]):
        
        if (step + update) % 100 != 0:
            return
        
        # get the counterfactual and target
        Y_t = sample["Y_t"]
        X_cf = model.params["decoder_inv_preprocessor"](sample["X_cf"])
        
        # get prediction label
        Y_m_cf = predictor(X_cf)
        
        # compute reward
        reward = np.mean(model.params["reward_func"](Y_m_cf, Y_t))
        wandb.log({"reward": reward})

#### Logging losses callback

In [24]:
class LossCallback(Callback):
    def __call__(self,
                 step: int, 
                 update: int, 
                 model: CounterfactualRL,
                 sample: Dict[str, np.ndarray],
                 losses: Dict[str, float]):
        # Log training losses.
        if (step + update) % 100 == 0:
            wandb.log(losses)

#### Logging tables callback

In [25]:
class TablesCallback(Callback):
    def __call__(self,
                 step: int, 
                 update: int, 
                 model: CounterfactualRL,
                 sample: Dict[str, np.ndarray],
                 losses: Dict[str, float]):
        # Log every 1000 steps
        if step % 1000 != 0:
            return
        
        # Define number of samples to be displayed.
        NUM_SAMPLES = 5
        
        X = heae_inv_preprocessor(sample["X"][:NUM_SAMPLES])        # input instance
        X_cf = heae_inv_preprocessor(sample["X_cf"][:NUM_SAMPLES])  # counterfactual
        
        Y_m = np.argmax(sample["Y_m"][:NUM_SAMPLES], axis=1).astype(int).reshape(-1, 1) # input labels
        Y_t = np.argmax(sample["Y_t"][:NUM_SAMPLES], axis=1).astype(int).reshape(-1, 1) # target labels
        Y_m_cf = np.argmax(predictor(X_cf), axis=1).astype(int).reshape(-1, 1)          # counterfactual labels
        
        # Define feature names and category map for input.
        feature_names = adult.feature_names + ["Label"]
        category_map = deepcopy(adult.category_map)
        category_map.update({feature_names.index("Label"): adult.target_names})
        
        # Construct input array.
        inputs = np.concatenate([X, Y_m], axis=1)
        inputs = pd.DataFrame(apply_category_mapping(inputs, category_map),
                              columns=feature_names)
        
        # Define feature names and category map for counterfactual output.
        feature_names += ["Target"]
        category_map.update({feature_names.index("Target"): adult.target_names})
        
        # Construct output array.
        outputs = np.concatenate([X_cf, Y_m_cf, Y_t], axis=1)
        outputs = pd.DataFrame(apply_category_mapping(outputs, category_map),
                               columns=feature_names)
        
        # Log table.
        wandb.log({
            "Input": wandb.Table(dataframe=inputs),
            "Output": wandb.Table(dataframe=outputs)
        })

Having defined the callbacks, we can define a new explainer that will include logging.

```python
import wandb

# Initialize wandb.
wandb_project = "Adult Census Counterfactual with Reinforcement Learning"
wandb.init(project=wandb_project)

# Define explainer as before and include callbacks.
explainer = CounterfactualRLTabular(...,
                                    callbacks=[LossCallback(), RewardCallback(), TablesCallback()])

# Fit the explainers.
explainer = explainer.fit(X=X_train)

# Close wandb.
wandb.finish()
```